In [1]:
# Function to perform pathfinding using the A* algorithm
def find_path(start, goal, grid):
    rows = len(grid)
    cols = len(grid[0])
    m=grid[goal[0]][goal[1]]
    grid[goal[0]][goal[1]]=0
    
    # Possible movements: up, down, left, right
    movements = [(0, -1), (0, 1), (-1, 0), (1, 0)]

    # Function to calculate the Manhattan distance between two points
    def heuristic(point1, point2):
        x1, y1 = point1
        x2, y2 = point2
        return abs(x2 - x1) + abs(y2 - y1)

    # Initialize the open and closed lists
    open_list = [start]
    closed_list = set()

    # Keep track of the parent of each point
    parents = {}

    # Cost from start to each point
    g_score = {start: 0}

    # Estimated total cost from start to goal through each point
    f_score = {start: heuristic(start, goal)}

    while open_list:
        current = min(open_list, key=lambda point: f_score[point])

        if current == goal:
            # Reconstruct the path
            path = [current]
            while current in parents:
                current = parents[current]
                path.append(current)
            path.reverse()
            grid[goal[0]][goal[1]]=m
            return path

        open_list.remove(current)
        closed_list.add(current)

        for dx, dy in movements:
            x, y = current
            neighbor = (x + dx, y + dy)

            if neighbor[0] < 0 or neighbor[0] >= rows or neighbor[1] < 0 or neighbor[1] >= cols:
                continue  # Ignore neighbors outside the grid

            if grid[neighbor[0]][neighbor[1]] != 0:
                continue  # Ignore neighbors that are obstacles

            if neighbor in closed_list:
                continue  # Ignore neighbors that have already been evaluated

            tentative_g_score = g_score[current] + 1

            if neighbor not in open_list or tentative_g_score < g_score[neighbor]:
                parents[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)

                if neighbor not in open_list:
                    open_list.append(neighbor)

    # No path found
    return []


In [2]:
# Function to calculate the Euclidean distance between two points
def calculate_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return (pow((x2 - x1)  ,2) + pow((y2 - y1) ,2)) ** 0.5

# Function to select the robot closest to a given room
def select_robot(room_location):
    closest_robot = None
    min_distance = float('inf')  # Initialize with a large value

    for robot in robots:
        if robot['available']:
            distance = calculate_distance(robot['location'], room_location)
            if distance < min_distance:
                min_distance = distance
                closest_robot = robot
                robot['available']=False
                return closest_robot['id']

In [3]:
import numpy as np

In [4]:
def available(robot,p):
    robots[robot-1]['battery_level']=100
    robots[robot-1]['available']=True
    print('\nrobot {} charged successfully\n'.format(robot))
    dechange(p)
    print(np.array(grid))
    robots[robot-1]['location']=(1,1)
    
def graph(path,robot):
    robots[robot-1]['available']=False
    change_grid(path,robot)
    print(np.array(grid))
    
def robot_charging_algorithm(robot,k,charge):
    charging_station = (1,1)
    if charge<=30:
        print('robot {} going to charge...\n'.format(robot))
        path=find_path(k,charging_station, grid)
        robots[robot-1]['available']=False
        return path


In [10]:
b=c=1
def func1(l):
    goal=rooms.get(l)
    c=0
    r=select_robot(goal)
    p=find_path(robots[r-1]['location'], goal, grid)
    print('robot {} in path to you...\n'.format(r))
    change_grid(p,r)
    robots[r-1]['available']=False
    print(np.array(grid))
    print('\n')
    return l,goal,c,r,p
    
    
def func2(l,goal,c,r,p):
    if c==0 and r!=None:
        robots[r-1]['battery_level']=robots[r-1]['battery_level']-len(p)*1
        print('robot {} in {}'.format(r,l))
        robots[r-1]['location']=rooms[l]
        dechange(p)
        print(np.array(grid))
        print('\n')
        goal=input('where your order ? ')
        if homes.get(goal)!=None:
            p=find_path(robots[r-1]['location'], homes[goal], grid)
            dechange([rooms[l]])
            change_grid(p,r)
            print(np.array(grid))
            print('\n')
        else:   
            print('not exist here\n')
            robots[r-1]['available']=True
            c=1
    return l,goal,c,r,p
    
    
def func3(l,goal,c,r,p):
    if c==0 and r!=None:
        robots[r-1]['battery_level']=robots[r-1]['battery_level']-len(p)*1
        robots[r-1]['location']=homes[goal]
        dechange(p)
        print('robot {} in {}\n'.format(r,goal))
        print(np.array(grid))
        print('\n')
        p=find_path(robots[r-1]['location'],rooms[l], grid)
        print('robot {} in path to you with your order... \n'.format(r))
        change_grid(p,r)
        print(np.array(grid))
        print('\n')
    return l,goal,c,r,p
    
    
def func4(l,goal,c,r,p):
    b=1
    if c==0 and r!=None:
        robots[r-1]['battery_level']=robots[r-1]['battery_level']-len(p)*1
        print('robot {} in {}'.format(r,l))
        robots[r-1]['location']=rooms[l]
        dechange(p)
        print(np.array(grid))
        print('\n')
        robots[r-1]['available']=True
        print('robot {} available now\n'.format(r))
        c=1
        p=robot_charging_algorithm(r,robots[r-1]['location'],robots[r-1]['battery_level'])
        if p!=None:
            graph(p,r)
            b=0
        
    return l,goal,c,r,p,b
    

In [18]:
rooms={'room1':(2,2),'room2':(2,5),'room4':(2,8),'room3':(2,11),'room5':(2,14)}
homes={'ph':(15,4),'cl1':(9,5),'cl2':(9,8),'nc':(9,11)}
robots = [
    {"id": 1, "location": (16,17), "available": True,"battery_level": 100},
    {"id": 2, "location": (12,17), "available": True,"battery_level": 100},
]
list1=[2,5,6,7,8,9,10,11,12,13]

def change_grid(p,m):
    for i in p:
        if (p[0] ==i or p[-1]==i) and grid[i[0]][i[1]]!=0:
             continue   
        grid[i[0]][i[1]]=m
        
def dechange(p):
    for i in p:
        if (p[0] ==i or p[-1]==i) and grid[i[0]][i[1]] not in[0,1,2]:
             continue 
        grid[i[0]][i[1]]=0

grid = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0,5, 0, 0,6, 0, 0,7, 0, 0,8, 0, 0,9, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 11, 0, 0, 12, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0,0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]


In [19]:
c1=c2=b1=b2=1
goal1=goal2=p1=p2=None


while(1):
    l1=input('name of room or exit ')
    if l1=='exit':
        print('closed')
        break
    elif rooms.get(l1)==None:
        r1=None
        print('error')
    else:
        l1,goal1,c1,r1,p1=func1(l1)
    l2=input('name of room or exit ')
    if l2=='exit':
        print('closed')
        break
    elif rooms.get(l2)==None:
        r2=None
        print('error')
    else:
        l2,goal2,c2,r2,p2=func1(l2)
    l1,goal1,c1,r1,p1=func2(l1,goal1,c1,r1,p1)
    l2,goal2,c2,r2,p2=func2(l2,goal2,c2,r2,p2)
    l1,goal1,c1,r1,p1=func3(l1,goal1,c1,r1,p1)
    l2,goal2,c2,r2,p2=func3(l2,goal2,c2,r2,p2)
    l1,goal1,c1,r1,p1,b1=func4(l1,goal1,c1,r1,p1)
    l2,goal2,c2,r2,p2,b2=func4(l2,goal2,c2,r2,p2)
    if b1==0:
        available(1,p1)
    if b2==0:
        available(2,p2)
    
    

name of room or exit room1
robot 1 in path to you...

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  6  0  0  7  0  0  8  0  0  9  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0 11  0  0 12  0  0 13  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 

name of room or exit room3
robot 1 in path to you...

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0  0]
 [ 0  0  5  1  1  6  0  0  7  0  0  8  0  0  9  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0 12  0  0 13  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 

name of room or exit exit
closed
